In [1]:
#import things to use
import os
import numpy as np
import tensorflow
import keras
from keras.models import Sequential
from keras.layers import Activation, Masking, Dense 
from keras.layers import Convolution2D as Conv2D 
from keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D 
from keras.layers import Softmax, ReLU, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from sklearn import metrics 
print('finished imports')

Using TensorFlow backend.


finished imports


In [2]:
#?
from keras.backend.tensorflow_backend import set_session
config = tensorflow.ConfigProto()
config.intra_op_parallelism_threads = 1
print(config.intra_op_parallelism_threads)
set_session(tensorflow.Session(config=config))

1


#import training data
#see getLabels_stackedData3.py for preprocessing

#change to the directory where list of STFT files is 
print(os.getcwd())
os.chdir('/home/ec2-user/SageMaker')
print(os.getcwd())

labels = []
allCols = []
allRows = []
allSTFTs = []
paddedSTFTs = []
maxTime = 0
print('made variables')

#open file (lists all train .npy STFT file names) 
file = open('TopList.txt', 'r') #note, ran with 169 in TopList, could not handle all 676
data = file.read().split('\n')
file.close()
print('got data files in a list')
print(data)

os.chdir('./aws_pilotwhales2')
print('changed to aws_pilotwhales2 folder')

for array in data:
    #loop through to find out if file is a buzz or minibuzz, and add label accordingly 
    nameParse = array.split("_",-1)
    #print(nameParse)
    typeParse = nameParse[4].split("u", -1)
    #print(typeParse)
    if(typeParse[0] == 'b'):
        #buzz = 1 
        labels.append(1)
    elif(typeParse[0] == 'minib'):
        #minibuzz = 0
        labels.append(0)
    else: 
        print('Error, not a buzz or minibuzz!')
    
    curSTFT = np.load(array)
        
    #find cols (number of time steps) of each STFT and save longest one
    rows, cols = curSTFT.shape
    allCols.append(cols)
    allRows.append(rows)
    if (cols>maxTime):
        maxTime = cols
    if (rows!=1025):
        print('Error, not 1025 STFT coefficients') 
        
    allSTFTs.append(curSTFT)
    
print(labels)
print(allRows)
print(allCols)
print(maxTime) 
print('first loop done')

for array in data: 
    #loop though STFTs again to zero pad, transpose, and reshape soo that there is one channel  
    #NOTE: must be done after we definitvely know the max number of time steps
    curSTFT = np.load(array)
    rows, cols = curSTFT.shape
    pad = maxTime-cols
    
    zeroPad = np.zeros((rows,maxTime-cols))
    paddedSTFT = np.append(curSTFT, zeroPad, axis = 1)
    
    paddedSTFT = np.transpose(paddedSTFT)
    paddedSTFT = np.reshape(paddedSTFT, paddedSTFT.shape + (1,))
    
    paddedSTFTs.append(paddedSTFT)

print('second loop done')

labels = np.array(labels)
#one hot encoding for labels
hotlabels = keras.utils.to_categorical(labels, num_classes=2, dtype='float32')
samples = np.array(paddedSTFTs)
print('hotlabels and samples in np arrays')

In [3]:
#before generator, loop through to find maxTime, Don't need to save anything else though

#change to the directory where list of STFT files is 
print(os.getcwd())
os.chdir('/home/ec2-user/SageMaker')
print(os.getcwd())

maxTime = 0
print('made variable')

#open file (lists all train .npy STFT file names) 
file = open('trainListALL.txt', 'r') #note, ran with 169 in TopList, could not handle all 676
data = file.read().split('\n')
file.close()
print('got data files in a list')
print(data)

os.chdir('/home/ec2-user/SageMaker/aws_pilotwhales2')
#print('changed to aws_pilotwhales2 folder')

for array in data:
    #find cols (number of time steps) of each STFT and save longest one
    curSTFT = np.load(array)
    rows, cols = curSTFT.shape
    if (cols>maxTime):
        maxTime = cols
    if (rows!=1025):
        print('Error, not 1025 STFT coefficients') 
print(maxTime) 
print('find max loop done!')

/home/ec2-user/SageMaker
/home/ec2-user/SageMaker
made variable
got data files in a list
['STFT_F_gm266a_event732_buzz124.npy', 'STFT_F_gm185b_event178_minibuzz28.npy', 'STFT_F_gm266a_event1202_buzz493.npy', 'STFT_F_gm185b_event199_minibuzz43.npy', 'STFT_F_gm267a_event1031_buzz276.npy', 'STFT_F_gm185b_event249_minibuzz74.npy', 'STFT_F_gm266a_event1204_buzz495.npy', 'STFT_F_gm208a_event443_minibuzz14.npy', 'STFT_F_gm267a_event1521_buzz387.npy', 'STFT_F_gm187b_event1602_minibuzz91.npy', 'STFT_F_gm266a_event1136_buzz439.npy', 'STFT_F_gm187b_event815_minibuzz62.npy', 'STFT_F_gm187b_event2612_buzz259.npy', 'STFT_F_gm185b_event222_minibuzz57.npy', 'STFT_F_gm267a_event343_buzz26.npy', 'STFT_F_gm208a_event240_minibuzz7.npy', 'STFT_F_gm208a_event666_buzz88.npy', 'STFT_F_gm187b_event1613_minibuzz96.npy', 'STFT_F_gm187b_event190_buzz36.npy', 'STFT_F_gm187b_event2588_minibuzz177.npy', 'STFT_F_gm266a_event1051_buzz367.npy', 'STFT_F_gm185b_event241_minibuzz69.npy', 'STFT_F_gm185b_event466_buzz16.npy

3559
find max loop done!


In [4]:
def data_generator(path, file, bs, maxTime, mode="train"):
    while True: 
        # open the text file for reading
        os.chdir(path)
        f = open(file, 'r')

        # initialize our batches of images and labels
        samples = []
        labels = []

        # keep looping until we reach our batch size
        while len(samples) < bs:
            # attempt to read the next line of the text file
            line = f.readline()
            #print(line)
            # check to see if the line is empty, indicating we have
            # reached the end of the file
            if line == "":
                # reset the file pointer to the beginning of the file
                # and re-read the line
                f.seek(0)
                line = f.readline()

            # extract the label and construct the sample
            #loop through to find out if file is a buzz or minibuzz, and add label accordingly 
            nameParse = line.split("_",-1)
            #print(nameParse)
            typeParse = nameParse[4].split("u", -1)
            #print(typeParse)
            if(typeParse[0] == 'b'):
                #buzz = 1 
                labels.append(1)
            elif(typeParse[0] == 'minib'):
                #minibuzz = 0
                labels.append(0)
            else: 
                print('Error, not a buzz or minibuzz!')

            #switch to aws_pilotwhales2 folder
            os.chdir('/home/ec2-user/SageMaker/aws_pilotwhales2')
            #print('changed to aws_pilotwhales2 folder')
            #print('text file name is:')
            #print(line)
            lineParse = line.split("\n",-1)
            #print(lineParse)
            #print(lineParse[0])
            #print(lineParse[1])
            curSTFT = np.load(lineParse[0])
            rows, cols = curSTFT.shape

            zeroPad = np.zeros((rows,maxTime-cols))
            paddedSTFT = np.append(curSTFT, zeroPad, axis = 1)

            paddedSTFT = np.transpose(paddedSTFT)
            paddedSTFT = np.reshape(paddedSTFT, paddedSTFT.shape + (1,))

            samples.append(paddedSTFT)

        labels = np.array(labels)
        #one hot encoding for labels
        hotlabels = keras.utils.to_categorical(labels, num_classes=2, dtype='float32')
        samples = np.array(samples)
        #print('hotlabels and samples in np arrays')

        # yield the batch to the calling function
        yield (samples, hotlabels)
        
# initialize both the training and testing image generators
trainGen = data_generator('/home/ec2-user/SageMaker', 'trainList.txt', 13, maxTime, mode = 'train')
print('generated training set!')
validGen = data_generator('/home/ec2-user/Sagemaker', 'validList.txt', 13, maxTime, mode = 'train')
print('generated vaildation set!')

generated training set!
generated vaildation set!


In [5]:
#layer 0: input
#labels[]
#paddedSTFTs[] (maxTime rows, 1025 cols) each

print('ready to start model')

# build model
model = Sequential()

print('made model')

#NOTE, CHANGED PADDING ON 2D CONVOLUTIONS from 'valid'=no paddinng to 'same'=padding so input and output are same dimensions

#layer 1: 2D convolution between input and 256 filters with 1 row and 1025 cols
model.add(Conv2D(256, input_shape = [maxTime,1025,1], kernel_size = [1,1025], strides=(1, 1), padding='valid', data_format="channels_last", dilation_rate=(1, 1), activation=None, use_bias=False, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None))
#batch normalization- add in layer? don't understand parameters well
#model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
#reLU layer
model.add(ReLU(max_value=None, negative_slope=0.0, threshold=0.0))
print('layer 1 done')

##layer 2: 2D convolution between output of layer 1 and 256 filters with 3 rows and 256 cols
model.add(Conv2D(256, kernel_size = [3,1], strides=(2, 1), padding='same', data_format="channels_last", dilation_rate=(1, 1), activation=None, use_bias=False, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None))
##batch normalization- add in layer? don't understand parameters well
##model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
##reLU layer
model.add(ReLU(max_value=None, negative_slope=0.0, threshold=0.0))
print('layer 2 done')

#layer 3: 2D convolution between output of layer 2 and 256 filters with 3 rows and 256 cols
model.add(Conv2D(256, kernel_size = [3,1], strides=(2, 1), padding='same', data_format="channels_last", dilation_rate=(1, 1), activation=None, use_bias=False, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None))
##batch normalization- add in layer? don't understand parameters well
##model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
##reLU layer
model.add(ReLU(max_value=None, negative_slope=0.0, threshold=0.0))
print('layer 3 done')

#layer 4: 2D convolution between output of layer 3 and 256 filters with 3 rows and 256 cols
model.add(Conv2D(256, kernel_size = [3,1], strides=(2, 1), padding='same', data_format="channels_last", dilation_rate=(1, 1), activation=None, use_bias=False, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None))
##batch normalization- add in layer? don't understand parameters well
##model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
##reLU layer
model.add(ReLU(max_value=None, negative_slope=0.0, threshold=0.0))
print('layer 4 done')

#layer 5: Global max pooling
model.add(GlobalMaxPooling2D(data_format="channels_last"))
print('layer 5 done')

#layer 6: fully connected layer
model.add(Dense(2, activation='softmax', use_bias=False, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None))
print('layer 6 done')

#Compile model [COMPILE]
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'], loss_weights=None, sample_weight_mode=None, weighted_metrics=None, target_tensors=None)
print('compiled!')

print(model.summary())

ready to start model
made model
layer 1 done
layer 2 done
layer 3 done
layer 4 done
layer 5 done
layer 6 done
compiled!
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 3559, 1, 256)      262400    
_________________________________________________________________
re_lu_1 (ReLU)               (None, 3559, 1, 256)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1780, 1, 256)      196608    
_________________________________________________________________
re_lu_2 (ReLU)               (None, 1780, 1, 256)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 890, 1, 256)       196608    
_________________________________________________________________
re_lu_3 (ReLU)               (None, 890, 1, 256)       0         
______________________

In [6]:
#Now let us train our model [FIT]
ES = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, patience=4, verbose=2, mode='auto', baseline=None, restore_best_weights=True)

#with fit_generator
print("[INFO] training w/ generator...")
model.fit_generator(trainGen, steps_per_epoch=42, epochs=5, verbose=2, callbacks=[ES], validation_data=validGen, validation_steps=10, class_weight=None, max_queue_size=10, workers=1, use_multiprocessing=False, shuffle=True, initial_epoch=0)
print('ran fit_generator!')

#with fit
#model.fit(x=samples, y=hotlabels, batch_size=13, epochs=5, verbose=2, callbacks=[ES], validation_split=0.2, validation_data=None, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None)
#print('ran fit')

[INFO] training w/ generator...
Epoch 1/5


MemoryError: 

In [ ]:
#[EVALUATE]
#do for test data, not training data! 
#preprocess STFT data in TestData folder!
#see getLabels_stackedData3.py, and give labels
#results = model.evaluate(x=testSamples, y=testHotlabels, batch_size=26, verbose=1, sample_weight=None, steps=None)
#results = model.evaluate(x=samples, y=hotlabels, batch_size=26, verbose=1, sample_weight=None, steps=None)
#print(results)
#print(model.metrics_names)

In [ ]:
#[PREDICT] (w/TestData I kept aside as well)
#preprocess STFT data in TestData folder!
#see getLabels_stackedData3.py, but don't give labels
#hotlabels_pred = model.predict(samples, batch_size=26, verbose=1, steps=None)

In [ ]:
#see if prediction results are right (compare hot labels I generate with what the model guesses)
#print((np.max(abs(hotlabels-hotlabels_pred),axis=1)>.5))